<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [23]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install pandas
# !pip install nltk

In [1]:
import os
import re
import csv
import json
import hazm
import nltk
import pickle
import zipfile
import requests
# import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [2]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [4]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [5]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [3]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [4]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [5]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]


def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم.
    </p>
</div>

In [6]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


preprocessor = Preprocessor(stopwords_path='stopwords.txt')

# texts = [data_to_text(data) for _, data in dataset.iterrows()]
# preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
# save_preprocessed_texts(preprocessed_texts)

preprocessed_texts = load_preprocessed_texts()

# Models
## Boolean

In [28]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts

In [38]:
class BooleanIR:

    def __init__(self):
        self.vectorizer = CountVectorizer(binary=True)
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def predict(self, query, dataset, k):
        query_transform = self.vectorizer.transform([query]).todense().tolist()[0]
        query_indices = np.nonzero(query_transform)[0]
        query_result = []
        for index, doc in self.dense_vectors_df.iterrows():
            if all(np.take(doc, query_indices)):
                query_result.append(index)
            if len(query_result) == k:
                break
        return dataset.iloc[query_result]


def save_boolean_model(model, path="BooleanIR_model.pickle"):
    with open(path, "wb") as file:
        pickle.dump(model, file)


def load_boolean_model(path="BooleanIR_model.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

In [39]:
mini_dataset, mini_preprocessed_texts = get_mini_dataset()

booleanIR_model = BooleanIR()
booleanIR_model.fit_transform_vectorizer(mini_preprocessed_texts)
save_boolean_model(booleanIR_model)

# booleanIR_model = load_boolean_model()

100%|██████████| 4000/4000 [00:22<00:00, 177.74it/s]


In [37]:
query = "روسیه به اوکراین حمله کرد"
preprocessed_query = ' '.join(preprocessor.preprocess(query))
booleanIR_model.predict(preprocessed_query, mini_dataset, k=10)

,date,title,intro,body,category
155,چهارشنبه ۱۷ فروردین ۱۴۰۱ - ۰۸:۴۵,فرید زکریا: اقدامات دولت بایدن دشمنان آمریکا ر...,فرید زکریا تحلیلگر ارشد آمریکایی در روزنامه وا...,به گزارش همشهری آنلاین به نقل از کیهان، یکی از...,Politics
266,شنبه ۷ اسفند ۱۴۰۰ - ۰۷:۴۶,با این روند پوتین مرد شماره یک جهان می‌شود | غ...,یک کارشناس مسائل بین الملل و استاد دانشگاه گفت...,محمد بیاتی - همشهری آنلاین: علی بیگدلی معتقد ا...,Politics
296,شنبه ۲۰ فروردین ۱۴۰۱ - ۰۸:۱۴,روایت نیویورک‌تایمز از دلیل حضور نداشتن آمریکا...,روزنامه نیویورک‌تایمز تأکید کرد: آمریکا در مذا...,به گزرش همشهری آنلاین به نقل از کیهان، این روز...,Politics
404,پنجشنبه ۱۵ اردیبهشت ۱۴۰۱ - ۰۳:۳۵,سوئد می‌گوید برای پیوستن به ناتو از آمریکا تضم...,وزیر خارجه سوئد گفت آمریکا به این کشور تضمین د...,به گزارش همشهری آنلاین به نقل از رویترز آنا لی...,World
416,یکشنبه ۱۴ فروردین ۱۴۰۱ - ۰۲:۵۱,پاپ فرانسیس ممکن است به کی‌یف سفر کند| انتقاد ...,پاپ فرانسیس گفته است در حال بررسی دیدار از شهر...,به گزارش همشهری آنلاین به نقل از گاردین رئیس ک...,World
418,سه‌شنبه ۱۶ فروردین ۱۴۰۱ - ۱۸:۳۵,اتحادیه اروپا دور پنجم تحریم‌ها بر ضد روسیه را...,اوزولا فون در لاین، رئیس کمیسیون اروپا اعلام ک...,به گزارش همشهری آنلاین به نقل از گاردین اوزولا...,World
420,پنجشنبه ۱۹ اسفند ۱۴۰۰ - ۱۴:۵۱,الیگارش‌های روس در انگلیس تحریم می‌شوند| آبرام...,دولت انگلیس رومن آبراموویچ، صاحب باشگاه فوتبال...,به گزارش همشهری آنلاین به نقل از گاردین وزارت ...,World
421,دوشنبه ۵ اردیبهشت ۱۴۰۱ - ۰۱:۴۹,اوکراین می‌گوید نیروهای روسیه قصد تصرف کارخانه...,مقامات اوکراینی روز یکشنبه گفتند نیروهای روسیه...,به گزارش همشهری آنلاین به نقل از رویترز فرماند...,World
424,جمعه ۱۲ فروردین ۱۴۰۱ - ۱۲:۲۷,انفجار در انبار نفت بلگورود روسیه | فرماندار: ...,فرماندار منطقه بلگورود روسیه مدعی شد که بالگرد...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World
433,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۴:۳۵,رئیس‌ جهموری جدید کره‌جنوبی را بهتر بشناسیم | ...,یون سوک یول، رئیس‌جمهوری کره‌جنوبی اگرچه در عر...,به گزارش همشهری‌ آنلاین و به نقل از فرانس۲۴، ی...,World


## TF-IDF

In [40]:
class TF_IDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def predict(self, query, dataset, k):
        query_transform = self.vectorizer.transform([query]).todense().tolist()[0]
        dense_vectors = self.dense_vectors_df.values.tolist()
        df_cosine_sim = list(map(lambda doc: self.cosine_sim(query_transform, doc), dense_vectors))
        self.dense_vectors_df['query_sim'] = df_cosine_sim
        indices = self.dense_vectors_df.nlargest(k, 'query_sim').index
        self.dense_vectors_df = self.dense_vectors_df.drop(columns=['query_sim'])
        return dataset.iloc[indices]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))


def save_TF_IDF_model(model, path="TF_IDF_model.pickle"):
    with open(path, "wb") as file:
        pickle.dump(model, file)


def load_TF_IDF_model(path="TF_IDF_model.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


In [41]:
mini_dataset, mini_preprocessed_texts = get_mini_dataset()

TF_IDF_model = TF_IDF()
TF_IDF_model.fit_transform_vectorizer(mini_preprocessed_texts)
save_TF_IDF_model(TF_IDF_model)

# TF_IDF_model = load_TF_IDF_model()

100%|██████████| 4000/4000 [00:21<00:00, 189.17it/s]


In [42]:
query = "روسیه به اوکراین حمله کرد"
preprocessed_query = ' '.join(preprocessor.preprocess(query))
TF_IDF_model.predict(preprocessed_query, mini_dataset, k=10)

,date,title,intro,body,category
733,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۰:۲۸,ببینید | اوکراین؛ از آژیر حمله هوایی تا آتش سو...,رئیس جمهوری روسیه صبح امروز اعلام کرد که در پا...,به گزارش همشهری آنلاین به نقل از تسنیم، شورای ...,World
501,یکشنبه ۱ اسفند ۱۴۰۰ - ۰۸:۴۹,هشدار درباره نزدیکی جنگ جهانی سوم | روسیه قصد ...,بوریس جانسون، نخست وزیر انگلیس هشدار داد که وض...,به گزارش همشهری آنلاین به نقل از ایسنا، جانسون...,World
444,دوشنبه ۲۵ بهمن ۱۴۰۰ - ۰۸:۲۴,روسیه ژست حمله به اوکراین را گرفت | شاید چین ه...,یک استاد ژئوپلیتیک دانشگاه با اشاره به اینکه ر...,به گزلرش همشهری آنلاین، عبدالرضا فرجی راد دربا...,World
3271,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
571,یکشنبه ۲۴ بهمن ۱۴۰۰ - ۱۱:۲۹,حمله قریب‌الوقوع روسیه به اوکراین؟,یک مقام کاخ الیزه می‌گوید در تماس تلفنی رئیس ج...,به گزارش همشهری آنلاین به نقل از فارس، یک مقام...,World
760,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۳:۱۱,تازه‌ترین درخواست وزیر امور خارجه اوکراین علیه...,دیمیترو کولبا، وزیر امور خارجه اوکراین، پس از ...,به گزارش همشهری آنلاین و به نقل از گاردین، او ...,World
764,جمعه ۶ اسفند ۱۴۰۰ - ۱۲:۱۹,ادعای وزیر دفاع انگلیس درباره نقشه روسیه برای ...,وزیر دفاع انگلیس در بیانیه‌ای مدعی شد، روسیه ق...,به گزارش همشهری آنلاین و به نقل از گاردین، بن ...,World
578,جمعه ۶ اسفند ۱۴۰۰ - ۱۶:۰۷,پوتین: آماده مذاکره با اوکراین هستیم | موضع ری...,رئیس جمهور چین در یک تماس تلفنی با رئیس جمهور ...,به گزارش همشهری‌آنلاین به نقل از فارس، رسانه‌ه...,World
589,شنبه ۱۷ اردیبهشت ۱۴۰۱ - ۱۶:۲۷,جنگ اوکراین روزانه چقدر برای روسیه آب می‌خورد؟,یک نشریه آمریکایی در گزارشی با اشاره به سومین ...,به گزارش همشهری‌آنلاین، ایرنا به نقل از نشریه ...,World
460,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۶:۱۰,منتظر جنگ جهانی سوم باشیم؟ | امکان چریکی شدن ج...,جهانگیر کرمی، عضو هیات علمی دانشگاه تهران و کا...,همشهری‌آنلاین - اصغر صوفی: درگیری میان اوکراین...,World


## Fasttext

In [44]:
class FastText:

    def __init__(self, method='skipgram'):
        self.method = method
        self.mean_embed = []
        self.model = None

    def train(self, dataset):
        with open('fasttext_train.txt', 'w', encoding='utf-8') as file:
            file.write('\n'.join(list(map(lambda doc: ' '.join(doc), dataset))))
        self.model = fasttext.train_unsupervised('fasttext_train.txt', self.method, minn=2, maxn=5)
        os.remove('fasttext_train.txt')
        for doc in dataset:
            embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), doc)), axis=0)
            self.mean_embed.append(embed)
        self.mean_embed = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = []
        for doc in self.mean_embed:
            doc_cosine_sim = self.cosine_sim(query_embed, doc)
            dataset_sim.append(doc_cosine_sim)
        idx = np.argsort(-np.array(dataset_sim))
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)
        np.save('FastText_mean_embed.npy', self.mean_embed)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('FastText_mean_embed.npy')

In [45]:
FastText_model = FastText()
FastText_model.train(preprocessed_texts)

NameError: name 'fasttext' is not defined

In [21]:
#FastText_model.save_ft_model()
FastText_model.load_ft_model()

In [32]:
query = "روسیه به اوکراین حمله کرد"
preprocessed_query = ' '.join(preprocessor.preprocess(query))
FastText_model.predict(preprocessed_query, dataset, k=10)

,date,title,intro,body,category-PER,category-ENG
57068,دوشنبه ۹ خرداد ۱۴۰۱ - ۰۶:۴۹,پیام تبریک سرمربی میلان ایتالیا به مجیدی و است...,سرمربی میلان به فرهاد مجیدی و بازیکنان استقلال...,به گزارش همشهری آنلاین استفانو پیولی سرمربی تی...,"['ورزش', 'فوتبال ايران']",Sport
49338,سه‌شنبه ۳ فروردین ۱۴۰۰ - ۰۹:۲۰,عکس | سلفی مجیدی با عضو جدید کادر فنی استقلال,همکار سرمربی استقلال در تیم ملی امید به کادرفن...,به گزارش همشهری‌آنلاین، فرشاد ماجدی به کادرفنی...,"['ورزش', 'فوتبال ايران']",Sport
72280,جمعه ۲۹ مرداد ۱۴۰۰ - ۲۲:۳۲,دستیار ایتالیایی مجیدی از ابتدای لیگ روی نیمکت...,گابریل پین با درخواست دوباره مجیدی به ایران خو...,به گزارش همشهری آنلاین یکی از خواسته های فرهاد...,"['ورزش', 'فوتبال ايران']",LifeSkills
51162,جمعه ۲۹ مرداد ۱۴۰۰ - ۲۲:۳۲,دستیار ایتالیایی مجیدی از ابتدای لیگ روی نیمکت...,گابریل پین با درخواست دوباره مجیدی به ایران خو...,به گزارش همشهری آنلاین یکی از خواسته های فرهاد...,"['ورزش', 'فوتبال ايران']",Sport
56582,یکشنبه ۲۵ اردیبهشت ۱۴۰۱ - ۱۳:۵۱,ناراحتی برانکو از قهرمان نشدن پرسپولیس در لیگ ...,سرمربی سابق کروات پرسپولیس به قهرمان نشدن پرسپ...,به گزارش همشهری‌آنلاین، سرمربی کروات و سابق پر...,"['ورزش', 'فوتبال ايران']",Sport
